# U/R + Λ Framework Demo

Mach–Zehnder + Bell/CHSH skeleton notebook.


## Mach–Zehnder in U/R language

We model a single-photon Mach–Zehnder interferometer as a 2D path space
$\mathcal{H}_U = \mathrm{span}\{|1\rangle, |2\rangle\}$, with a first beam splitter,
a phase shifter $\phi$ in arm 2, and a second beam splitter. Sector R only records
which detector clicked.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Beam splitter (50/50)
B = (1/np.sqrt(2)) * np.array([[1, 1],
                               [1, -1]])

def P_phi(phi):
    """Phase shifter: phase on path |2⟩."""
    return np.array([[1, 0],
                     [0, np.exp(1j*phi)]])

# Initial state: photon in input 1
psi_in = np.array([1, 0], dtype=complex)

def mz_output_amplitudes(phi):
    """Apply BS1 → P(phi) → BS2 to psi_in."""
    psi1 = B @ psi_in           # after first BS
    psi2 = P_phi(phi) @ psi1    # after phase shifter
    psi_out = B @ psi2          # after second BS
    return psi_out  # [amp at D0, amp at D1]

phis = np.linspace(0, 2*np.pi, 200)
P_D0 = []
P_D1 = []

for phi in phis:
    amp = mz_output_amplitudes(phi)
    P_D0.append(np.abs(amp[0])**2)
    P_D1.append(np.abs(amp[1])**2)

P_D0 = np.array(P_D0)
P_D1 = np.array(P_D1)

plt.figure(figsize=(6,4))
plt.plot(phis, P_D0, label="P(D0)")
plt.plot(phis, P_D1, label="P(D1)")
plt.xlabel("φ")
plt.ylabel("Probability")
plt.title("Mach–Zehnder output probabilities (Sector U)")
plt.legend()
plt.grid(True)
plt.show()


## Bell/CHSH test in U/R language

We consider a standard Bell pair shared by Alice and Bob. Sector U holds
the two-qubit state and measurement operators; Sector R records their
settings and outcomes. We compute the CHSH value using standard
quantum operators.


In [ ]:
import numpy as np

# Pauli matrices
sx = np.array([[0, 1],
               [1, 0]], dtype=complex)
sy = np.array([[0, -1j],
               [1j, 0]], dtype=complex)
sz = np.array([[1, 0],
               [0, -1]], dtype=complex)
I2 = np.eye(2, dtype=complex)

def kron(a, b):
    return np.kron(a, b)

# Bell state |Phi+> = (|00> + |11>)/sqrt(2)
phi_plus = (1/np.sqrt(2)) * np.array([1, 0, 0, 1], dtype=complex)

def expectation(state, operator):
    return np.vdot(state, operator @ state).real

# Measurement settings
# Alice: A0 = sz, A1 = sx
# Bob:   B0 = (sz + sx)/sqrt(2), B1 = (sz - sx)/sqrt(2)
A0 = sz
A1 = sx
B0 = (sz + sx) / np.sqrt(2)
B1 = (sz - sx) / np.sqrt(2)

# Correlators E(Ai,Bj)
E00 = expectation(phi_plus, kron(A0, B0))
E01 = expectation(phi_plus, kron(A0, B1))
E10 = expectation(phi_plus, kron(A1, B0))
E11 = expectation(phi_plus, kron(A1, B1))

S = E00 + E01 + E10 - E11
print("E00, E01, E10, E11 =", E00, E01, E10, E11)
print("CHSH S =", S)
